# Example: Radar processing

The processing of radar data - often referred to as SAR data, which stands for Sythetice Aperture Radar - is a topic of its own and quite different from optical data that uses Red, Green, Blue, NIR and similar bands.
This is the first of a series of jupyter notebooks where example SAR workflows will be explained that can be run on UP42.
All of the examples are based on a specific type of analysis which is called polarimetry and processes one image at a time.
In comparison to polarimetric there also is interferometric analysis which leverages the information encoded in suitable pairs of radar image. 

In this notebook we want to have a look at the Sentinel-1 GRD full scene block, followed by SNAP Sentinel-1 Polarimetric Processing.

 - Prepare workflow
 - Define aoi and input parameters
 - Run Job
 - Visualize results

In [ ]:
import up42

In [ ]:
# Authenticate with UP42
up42.authenticate(cfg_file="config.json", env="dev")
#up42.authenticate(project_id=12345, project_api_key=12345)

## Prepare UP42 workflows

Create a new project on UP42 or use an existing one.

In [ ]:
project = up42.initialize_project()
S1_SNAP_project = up42.initialize_project()
project

Create workflow and check available blocks and data

In [ ]:
workflow = S1_SNAP_project.create_workflow(name="S1-GRD_SNAP", 
                                   use_existing=True)
print(up42.get_blocks(basic=True))

In [ ]:
# Fill the workflow with tasks
input_tasks= ['sobloo-s1-grd-fullscene', 'snap-polarimetric']
workflow.add_workflow_tasks(input_tasks=input_tasks)
workflow.get_parameters_info()

## Define aoi and input parameters 

In [ ]:
input_parameters = {
  "sobloo-s1-grd-fullscene:1": {
    "bbox": [13.371037, 52.512799, 13.382624, 52.518747],
    "ids": None,
    "time": "2018-01-01T00:00:00+00:00/2020-12-31T23:59:59+00:00",
    "limit": 1,
    "zoom_level": 14,
  },
    "snap-polarimetric:1": {
    "bbox": [13.371037, 52.512799, 13.382624, 52.518747],
    "mask": None,
    "contains": None,
    "intersects": None,
    "clip_to_aoi": True,
    "tcorrection": True,
    "linear_to_db": True,
    "polarisations": [
        "VV"
   ],
   "speckle_filter": True,
   "calibration_band": [
         "sigma"
   ]
  }
}

## Run test query
Run a test job to query data availability and check the configuration.

In [ ]:
test_job = workflow.test_job(input_parameters=input_parameters, track_status=True)
test_results = test_job.get_results_json()
print(test_results)

## Run the job

In [ ]:
job = workflow.run_job(input_parameters=input_parameters, track_status=True)

## Visualize the results

In [ ]:
# Download results:
job.download_results()

# Visualize downloaded results
job.plot_results()